In [20]:
import os
import pandas as pd
import ast
import numpy as np

# -------------------- 1. 공통 함수 정의 --------------------

def parse_list(x):
    """문자열을 리스트로 변환"""
    return ast.literal_eval(x)

def sum_int_list(x):
    """리스트 내부를 int로 변환 후 sum 계산"""
    return sum([int(v) for v in x]) if len(x) > 0 else 0

def trim_list(x, n, is_numeric=False):
    """앞 n개 리스트 자르기 → 숫자형이면 int 변환"""
    if is_numeric:
        return str([int(v) for v in x[:n]])
    else:
        return str(x[:n])

# -------------------- 2. 설정 --------------------

folder_path = '../../data'
csv_filename = 'make_test_data.csv'
csv_path = os.path.join(folder_path, csv_filename)

output_filename = 'test_data_10.csv'
output_path = os.path.join(folder_path, output_filename)

N_TOP_VALUES = 10

# -------------------- 3. 데이터 불러오기 --------------------

df = pd.read_csv(csv_path, encoding="utf-8")

# -------------------- 4. 리스트 컬럼 정의 --------------------

columns_to_extract = [
    '조회일', '순위', '매출액', '총매출액', '매출액(점유율)', '매출액증감',
    '매출액증감율(전일대비)', '관객수', '관객수증감(전일대비)', '관객수증감율(전일대비)', '스크린수', '상영횟수'
]

list_like_columns = [
    col for col in columns_to_extract
    if isinstance(df[col].iloc[0], str) and df[col].iloc[0].strip().startswith('[') and df[col].iloc[0].strip().endswith(']')
]

numeric_list_columns = [
    '매출액', '매출액증감', '관객수', '관객수증감(전일대비)', '스크린수', '상영횟수'
]

# -------------------- 5. 총매출액/누적관객수 계산 --------------------

# 미리 리스트 변환
매출액_list_values = df['매출액'].apply(parse_list)
관객수_list_values = df['관객수'].apply(parse_list)

# 계산 결과 시리즈 생성
총매출액_series = 매출액_list_values.apply(sum_int_list)
누적관객수_series = 관객수_list_values.apply(sum_int_list)

# -------------------- 6. 최종 result_df 구성 --------------------

result_df = pd.DataFrame()

for col in df.columns:
    if col in list_like_columns:
        list_values = df[col].apply(parse_list)
        is_numeric = col in numeric_list_columns
        result_df[col] = list_values.apply(lambda x: trim_list(x, N_TOP_VALUES, is_numeric=is_numeric))
    
    elif col == '총매출액':
        result_df[col] = 총매출액_series
    
    elif col == '누적관객수':
        result_df[col] = 누적관객수_series
    
    else:
        result_df[col] = df[col]

# -------------------- 7. 저장 --------------------

result_df.to_csv(output_path, index=False, encoding="utf-8-sig")

# -------------------- 8. 결과 확인 --------------------

print(f"저장 완료: {output_path}")
print(result_df.head())
print("컬럼 순서:", result_df.columns.tolist())


저장 완료: ../../data\test_data_10.csv
          영화명         개봉일                장르  국적         감독  \
0     너의 이름은.  2017-01-04  애니메이션,드라마,멜로/로맨스  일본    신카이 마코토   
1      날씨의 아이  2019-10-30  애니메이션,판타지,멜로/로맨스  일본    신카이 마코토   
2  더 퍼스트 슬램덩크  2023-01-04             애니메이션  일본  이노우에 다케히코   
3        라라랜드  2016-12-07    드라마,뮤지컬,멜로/로맨스  미국    데이미언 셔젤   
4  극장판 주술회전 0  2022-02-17             애니메이션  일본        박성후   

                                        배급사 대표국적        등급  제작사  \
0                     메가박스중앙(주) 플러스엠 엔터테인먼트   일본  12세이상관람가  NaN   
1  메가박스중앙(주) 플러스엠 엔터테인먼트,(주)미디어캐슬,워터홀컴퍼니(주)   일본  15세이상관람가  NaN   
2                       (주)넥스트엔터테인먼트월드(NEW)   일본  12세이상관람가  NaN   
3                                   판씨네마(주)   미국  12세이상관람가  NaN   
4              (주)바이포엠스튜디오,(주)대교 미디어콘텐츠사업본부   일본  15세이상관람가  NaN   

                                                  배우  ...  \
0  카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...  ...   
1                 다이고 코타로,모리 나나,오구리 슌,심규혁,최한,강은애,김서영  ...   
2     

In [21]:
import os
import pandas as pd
import ast

# -------------------- 공통 함수 정의 --------------------

def parse_list(x):
    """문자열을 리스트로 변환"""
    return ast.literal_eval(x)

def sum_int_list(x):
    """리스트 내부를 int로 변환 후 sum 계산"""
    return sum([int(v) for v in x]) if len(x) > 0 else 0

# -------------------- 설정 --------------------

# 파일 경로 설정
folder_path = '../../data'
csv_filename = 'test_data_10.csv'
csv_path = os.path.join(folder_path, csv_filename)

# 결과 파일명
output_filename = 'test_data_10.csv'
output_path = os.path.join(folder_path, output_filename)

# -------------------- 데이터 불러오기 --------------------

df = pd.read_csv(csv_path, encoding='utf-8')

# -------------------- 총매출액/누적관객수 재계산 --------------------

# 매출액 리스트 변환 후 sum 계산
df['총매출액'] = df['매출액'].apply(lambda x: sum_int_list(parse_list(x)))

# 관객수 리스트 변환 후 sum 계산
df['누적관객수'] = df['관객수'].apply(lambda x: sum_int_list(parse_list(x)))

# -------------------- 결과 저장 --------------------

df.to_csv(output_path, index=False, encoding='utf-8-sig')

# -------------------- 결과 확인 --------------------

print(f"저장 완료: {output_path}")
print(df[['총매출액', '누적관객수']].head())


저장 완료: ../../data\test_data_10.csv
         총매출액   누적관객수
0     1536280     304
1     6879260    1326
2  1475211475  139771
3    50062260    9655
4   516647410   47731


In [22]:
import os
import pandas as pd
import ast
import numpy as np

# -------------------- 1. 공통 함수 정의 --------------------

def parse_list(x):
    """문자열을 리스트로 변환"""
    return ast.literal_eval(x)

def sum_int_list(x):
    """리스트 내부를 int로 변환 후 sum 계산"""
    return sum([int(v) for v in x]) if len(x) > 0 else 0

def trim_list(x, n, is_numeric=False):
    """앞 n개 리스트 자르기 → 숫자형이면 int 변환"""
    if is_numeric:
        return str([int(v) for v in x[:n]])
    else:
        return str(x[:n])

# -------------------- 2. 설정 --------------------

folder_path = '../../data'
csv_filename = 'make_test_data.csv'
csv_path = os.path.join(folder_path, csv_filename)

output_filename = 'test_data_7.csv'
output_path = os.path.join(folder_path, output_filename)

N_TOP_VALUES = 7

# -------------------- 3. 데이터 불러오기 --------------------

df = pd.read_csv(csv_path, encoding="utf-8")

# -------------------- 4. 리스트 컬럼 정의 --------------------

columns_to_extract = [
    '조회일', '순위', '매출액', '총매출액', '매출액(점유율)', '매출액증감',
    '매출액증감율(전일대비)', '관객수', '관객수증감(전일대비)', '관객수증감율(전일대비)', '스크린수', '상영횟수'
]

list_like_columns = [
    col for col in columns_to_extract
    if isinstance(df[col].iloc[0], str) and df[col].iloc[0].strip().startswith('[') and df[col].iloc[0].strip().endswith(']')
]

numeric_list_columns = [
    '매출액', '매출액증감', '관객수', '관객수증감(전일대비)', '스크린수', '상영횟수'
]

# -------------------- 5. 총매출액/누적관객수 계산 --------------------

# 미리 리스트 변환
매출액_list_values = df['매출액'].apply(parse_list)
관객수_list_values = df['관객수'].apply(parse_list)

# 계산 결과 시리즈 생성
총매출액_series = 매출액_list_values.apply(sum_int_list)
누적관객수_series = 관객수_list_values.apply(sum_int_list)

# -------------------- 6. 최종 result_df 구성 --------------------

result_df = pd.DataFrame()

for col in df.columns:
    if col in list_like_columns:
        list_values = df[col].apply(parse_list)
        is_numeric = col in numeric_list_columns
        result_df[col] = list_values.apply(lambda x: trim_list(x, N_TOP_VALUES, is_numeric=is_numeric))
    
    elif col == '총매출액':
        result_df[col] = 총매출액_series
    
    elif col == '누적관객수':
        result_df[col] = 누적관객수_series
    
    else:
        result_df[col] = df[col]

# -------------------- 7. 저장 --------------------

result_df.to_csv(output_path, index=False, encoding="utf-8-sig")

# -------------------- 8. 결과 확인 --------------------

print(f"저장 완료: {output_path}")
print(result_df.head())
print("컬럼 순서:", result_df.columns.tolist())


저장 완료: ../../data\test_data_7.csv
          영화명         개봉일                장르  국적         감독  \
0     너의 이름은.  2017-01-04  애니메이션,드라마,멜로/로맨스  일본    신카이 마코토   
1      날씨의 아이  2019-10-30  애니메이션,판타지,멜로/로맨스  일본    신카이 마코토   
2  더 퍼스트 슬램덩크  2023-01-04             애니메이션  일본  이노우에 다케히코   
3        라라랜드  2016-12-07    드라마,뮤지컬,멜로/로맨스  미국    데이미언 셔젤   
4  극장판 주술회전 0  2022-02-17             애니메이션  일본        박성후   

                                        배급사 대표국적        등급  제작사  \
0                     메가박스중앙(주) 플러스엠 엔터테인먼트   일본  12세이상관람가  NaN   
1  메가박스중앙(주) 플러스엠 엔터테인먼트,(주)미디어캐슬,워터홀컴퍼니(주)   일본  15세이상관람가  NaN   
2                       (주)넥스트엔터테인먼트월드(NEW)   일본  12세이상관람가  NaN   
3                                   판씨네마(주)   미국  12세이상관람가  NaN   
4              (주)바이포엠스튜디오,(주)대교 미디어콘텐츠사업본부   일본  15세이상관람가  NaN   

                                                  배우  ...  \
0  카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...  ...   
1                 다이고 코타로,모리 나나,오구리 슌,심규혁,최한,강은애,김서영  ...   
2      

In [23]:
import os
import pandas as pd
import ast

# -------------------- 공통 함수 정의 --------------------

def parse_list(x):
    """문자열을 리스트로 변환"""
    return ast.literal_eval(x)

def sum_int_list(x):
    """리스트 내부를 int로 변환 후 sum 계산"""
    return sum([int(v) for v in x]) if len(x) > 0 else 0

# -------------------- 설정 --------------------

# 파일 경로 설정
folder_path = '../../data'
csv_filename = 'test_data_7.csv'
csv_path = os.path.join(folder_path, csv_filename)

# 결과 파일명
output_filename = 'test_data_7.csv'
output_path = os.path.join(folder_path, output_filename)

# -------------------- 데이터 불러오기 --------------------

df = pd.read_csv(csv_path, encoding='utf-8')

# -------------------- 총매출액/누적관객수 재계산 --------------------

# 매출액 리스트 변환 후 sum 계산
df['총매출액'] = df['매출액'].apply(lambda x: sum_int_list(parse_list(x)))

# 관객수 리스트 변환 후 sum 계산
df['누적관객수'] = df['관객수'].apply(lambda x: sum_int_list(parse_list(x)))

# -------------------- 결과 저장 --------------------

df.to_csv(output_path, index=False, encoding='utf-8-sig')

# -------------------- 결과 확인 --------------------

print(f"저장 완료: {output_path}")
print(df[['총매출액', '누적관객수']].head())


저장 완료: ../../data\test_data_7.csv
        총매출액  누적관객수
0    1325280    261
1    5997360   1163
2  398842363  39586
3   44926840   8657
4  334298930  30876


In [24]:
import os
import pandas as pd
import ast
import numpy as np

# -------------------- 1. 공통 함수 정의 --------------------

def parse_list(x):
    """문자열을 리스트로 변환"""
    return ast.literal_eval(x)

def sum_int_list(x):
    """리스트 내부를 int로 변환 후 sum 계산"""
    return sum([int(v) for v in x]) if len(x) > 0 else 0

def trim_list(x, n, is_numeric=False):
    """앞 n개 리스트 자르기 → 숫자형이면 int 변환"""
    if is_numeric:
        return str([int(v) for v in x[:n]])
    else:
        return str(x[:n])

# -------------------- 2. 설정 --------------------

folder_path = '../../data'
csv_filename = 'make_test_data.csv'
csv_path = os.path.join(folder_path, csv_filename)

output_filename = 'test_data_3.csv'
output_path = os.path.join(folder_path, output_filename)

N_TOP_VALUES = 3

# -------------------- 3. 데이터 불러오기 --------------------

df = pd.read_csv(csv_path, encoding="utf-8")

# -------------------- 4. 리스트 컬럼 정의 --------------------

columns_to_extract = [
    '조회일', '순위', '매출액', '총매출액', '매출액(점유율)', '매출액증감',
    '매출액증감율(전일대비)', '관객수', '관객수증감(전일대비)', '관객수증감율(전일대비)', '스크린수', '상영횟수'
]

list_like_columns = [
    col for col in columns_to_extract
    if isinstance(df[col].iloc[0], str) and df[col].iloc[0].strip().startswith('[') and df[col].iloc[0].strip().endswith(']')
]

numeric_list_columns = [
    '매출액', '매출액증감', '관객수', '관객수증감(전일대비)', '스크린수', '상영횟수'
]

# -------------------- 5. 총매출액/누적관객수 계산 --------------------

# 미리 리스트 변환
매출액_list_values = df['매출액'].apply(parse_list)
관객수_list_values = df['관객수'].apply(parse_list)

# 계산 결과 시리즈 생성
총매출액_series = 매출액_list_values.apply(sum_int_list)
누적관객수_series = 관객수_list_values.apply(sum_int_list)

# -------------------- 6. 최종 result_df 구성 --------------------

result_df = pd.DataFrame()

for col in df.columns:
    if col in list_like_columns:
        list_values = df[col].apply(parse_list)
        is_numeric = col in numeric_list_columns
        result_df[col] = list_values.apply(lambda x: trim_list(x, N_TOP_VALUES, is_numeric=is_numeric))
    
    elif col == '총매출액':
        result_df[col] = 총매출액_series
    
    elif col == '누적관객수':
        result_df[col] = 누적관객수_series
    
    else:
        result_df[col] = df[col]

# -------------------- 7. 저장 --------------------

result_df.to_csv(output_path, index=False, encoding="utf-8-sig")

# -------------------- 8. 결과 확인 --------------------

print(f"저장 완료: {output_path}")
print(result_df.head())
print("컬럼 순서:", result_df.columns.tolist())


저장 완료: ../../data\test_data_3.csv
          영화명         개봉일                장르  국적         감독  \
0     너의 이름은.  2017-01-04  애니메이션,드라마,멜로/로맨스  일본    신카이 마코토   
1      날씨의 아이  2019-10-30  애니메이션,판타지,멜로/로맨스  일본    신카이 마코토   
2  더 퍼스트 슬램덩크  2023-01-04             애니메이션  일본  이노우에 다케히코   
3        라라랜드  2016-12-07    드라마,뮤지컬,멜로/로맨스  미국    데이미언 셔젤   
4  극장판 주술회전 0  2022-02-17             애니메이션  일본        박성후   

                                        배급사 대표국적        등급  제작사  \
0                     메가박스중앙(주) 플러스엠 엔터테인먼트   일본  12세이상관람가  NaN   
1  메가박스중앙(주) 플러스엠 엔터테인먼트,(주)미디어캐슬,워터홀컴퍼니(주)   일본  15세이상관람가  NaN   
2                       (주)넥스트엔터테인먼트월드(NEW)   일본  12세이상관람가  NaN   
3                                   판씨네마(주)   미국  12세이상관람가  NaN   
4              (주)바이포엠스튜디오,(주)대교 미디어콘텐츠사업본부   일본  15세이상관람가  NaN   

                                                  배우  ...  \
0  카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...  ...   
1                 다이고 코타로,모리 나나,오구리 슌,심규혁,최한,강은애,김서영  ...   
2      

In [25]:
import os
import pandas as pd
import ast

# -------------------- 공통 함수 정의 --------------------

def parse_list(x):
    """문자열을 리스트로 변환"""
    return ast.literal_eval(x)

def sum_int_list(x):
    """리스트 내부를 int로 변환 후 sum 계산"""
    return sum([int(v) for v in x]) if len(x) > 0 else 0

# -------------------- 설정 --------------------

# 파일 경로 설정
folder_path = '../../data'
csv_filename = 'test_data_3.csv'
csv_path = os.path.join(folder_path, csv_filename)

# 결과 파일명
output_filename = 'test_data_3.csv'
output_path = os.path.join(folder_path, output_filename)

# -------------------- 데이터 불러오기 --------------------

df = pd.read_csv(csv_path, encoding='utf-8')

# -------------------- 총매출액/누적관객수 재계산 --------------------

# 매출액 리스트 변환 후 sum 계산
df['총매출액'] = df['매출액'].apply(lambda x: sum_int_list(parse_list(x)))

# 관객수 리스트 변환 후 sum 계산
df['누적관객수'] = df['관객수'].apply(lambda x: sum_int_list(parse_list(x)))

# -------------------- 결과 저장 --------------------

df.to_csv(output_path, index=False, encoding='utf-8-sig')

# -------------------- 결과 확인 --------------------

print(f"저장 완료: {output_path}")
print(df[['총매출액', '누적관객수']].head())


저장 완료: ../../data\test_data_3.csv
       총매출액  누적관객수
0    541820    107
1   2614880    505
2  38637000   3621
3  21266660   4048
4  21743200   1748
